In [2]:
%load_ext autoreload

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import protfasta 
%autoreload 2
%aimport AD_predictor_tools
%aimport AD_comparison_tools
%aimport PlottingTools

In [21]:
# Loading in Known ADs
known_ADs = pd.read_csv("../output/known_ADs_considering_isoforms.csv")
known_ADs

,Gene,Start,End,uniprotID,Matching Isoforms,Canonical Transcript ID,orig_uniprotID,Reference,TileType,ProteinRegionSeq,Notes
0,ABRAXAS1,121,200,Q6UWZ7,[['Q6UWZ7']],ENST00000321945,Q6UWZ7,DelRosso et al.,CR,LQEHFSNQDLVFLLLTPSIITESCSTHRLEHSLYKPQKGLFHRVPL...,NaN
1,AEBP1,1088,1158,Q8IUX7,[['Q8IUX7']],NaN,Q8IUX7,Staller Activity Data,TF,EVVTEFGTEVEPEFGTKVEPEFETQLEPEFETQLEPEFEEEEEEEK...,NaN
2,AHCTF1,1445,1698,Q8WYP5,"[['Q8WYP5'], ['Q8WYP5']]",nan / nan,Q8WYP5 / Q8WYP5,"PMID: 11952839, Soto / transcriptionalactivity...",TF,IRANDNKSMADVLGDGGNSSLTISEGPIVSERRLNQEVALNLKEDH...,NaN
3,AHR,118,126,P35869,[['P35869']],NaN,P35869,"transcriptionalactivity_regions.txt, GSL",TF,LLQALNGFV,NaN
4,AHR,266,268,P35869,[['P35869']],NaN,P35869,"transcriptionalactivity_regions.txt, GSL",TF,FAI,NaN
...,...,...,...,...,...,...,...,...,...,...,...
755,ZXDB,576,703,P98169,[['P98169']],NaN,P98169 / P98169,"R4TA_regions.txt / activation_regions.txt, GSL",TF,QDLLAQLEAANSLTPSSELTSQRQNDLSDAEIVSLFSDVPDSTSAA...,NaN
756,ZXDC,579,688,Q2QGD7,"[['Q2QGD7', 'Q2QGD7-2'], ['Q2QGD7', 'Q2QGD7-2']]",nan / nan,Q2QGD7 / Q2QGD7 / Q2QGD7,"PMID: 16600381, Soto / R4TA_regions.txt / acti...",TF,DSPLVLGTAATVLQQGSFSVDDVQTVSAGALGCLVALPMKNLSDDP...,NaN
757,c-ETS1,175,286,P14921-3,[['P14921-3']],NaN,P14921,"Choi 2000 list, GSL",TF,ILQKEDVKPYQVNGVNPAYPESRYTSDYFISYGIEHAQCVPPSEFS...,NaN
758,c-Jun/AP-1,238,257,P05412,[['P05412']],NaN,P05412,"Choi 2000 list, GSL",TF,ETPPLSPIDMESQERIKAER,NaN


In [6]:
# Loading in SFARI genes  
SFARI = pd.read_csv("../data/SFARI-Gene_genes_01-23-2023release_03-21-2023export.csv")
SFARI

,status,gene-symbol,gene-name,ensembl-id,chromosome,genetic-category,gene-score,syndromic,eagle,number-of-reports
0,9,ABAT,4-aminobutyrate aminotransferase,ENSG00000183044,16,"Rare Single Gene Mutation, Genetic Association",2.0,0,NaN,7
1,9,ABCA10,"ATP-binding cassette, sub-family A (ABC1), mem...",ENSG00000154263,17,Rare Single Gene Mutation,2.0,0,NaN,2
2,9,ABCA13,ATP binding cassette subfamily A member 13,ENSG00000179869,7,"Rare Single Gene Mutation, Functional",2.0,0,NaN,10
3,9,ABCA7,"ATP-binding cassette, sub-family A (ABC1), mem...",ENSG00000064687,19,Rare Single Gene Mutation,2.0,0,NaN,5
4,9,ABL2,"ABL proto-oncogene 2, non-receptor tyrosine ki...",ENSG00000143322,1,"Rare Single Gene Mutation, Functional",3.0,0,NaN,10
...,...,...,...,...,...,...,...,...,...,...
1113,9,ZNF774,Zinc finger protein 774,ENSG00000196391,15,Rare Single Gene Mutation,2.0,0,NaN,4
1114,9,ZNF804A,Zinc finger protein 804A,ENSG00000170396,2,"Rare Single Gene Mutation, Genetic Association...",2.0,0,NaN,16
1115,9,ZNF827,Zinc finger protein 827,ENSG00000151612,4,"Rare Single Gene Mutation, Genetic Association",2.0,0,NaN,3
1116,9,ZSWIM6,zinc finger SWIM-type containing 6,ENSG00000130449,5,"Rare Single Gene Mutation, Syndromic, Genetic ...",NaN,1,NaN,6


In [7]:
SFARI[SFARI["ensembl-id"].isna()]

,status,gene-symbol,gene-name,ensembl-id,chromosome,genetic-category,gene-score,syndromic,eagle,number-of-reports
11,9,ADA,adenosine deaminase,NaN,20,"Rare Single Gene Mutation, Genetic Association",2.0,0,NaN,8
66,9,AR,androgen receptor,NaN,X,Genetic Association,2.0,0,NaN,6
559,9,MAPT-AS1,MAPT antisense RNA 1,NaN,17,Genetic Association,2.0,0,NaN,1
583,9,MET,met proto-oncogene (hepatocyte growth factor r...,NaN,7,"Rare Single Gene Mutation, Genetic Association...",2.0,0,NaN,40
588,9,MIR137,microRNA 137,NaN,1,"Rare Single Gene Mutation, Syndromic, Genetic ...",2.0,0,NaN,10
631,9,MSNP1AS,"Moesinpseudogene 1, antisense",NaN,5,"Genetic Association, Functional",2.0,0,NaN,13
792,9,PTCHD1-AS,PTCHD1antisense RNA (head to head),NaN,X,Rare Single Gene Mutation,2.0,0,17.6,3
847,9,RP11-1407O15.2,NaN,NaN,17,Rare Single Gene Mutation,2.0,0,NaN,1
871,9,RPS10P2-AS1,ribosomal protein S10 pseudogene 2 anti-sense 1,NaN,20,"Genetic Association, Functional",2.0,0,NaN,4


In [24]:
# Using uniprotIDs of known ADs to get gene names
known_ADs[["uniprotID"]].to_csv("../data/known_AD_uniprotIDs_for_gene_names.txt", 
                                                 header=None, index=None, sep=' ', mode='a')

In [8]:
known_AD_gene_names = pd.read_csv("../data/known_AD_gene_names.tsv", sep = "\t")
known_AD_gene_names

,From,Entry,Gene Names (primary)
0,Q6UWZ7,Q6UWZ7,ABRAXAS1
1,Q8IUX7,Q8IUX7,AEBP1
2,Q8WYP5,Q8WYP5,AHCTF1
3,P35869,P35869,AHR
4,O43918,O43918,AIRE
...,...,...,...
589,P98168,P98168,ZXDA
590,P98169,P98169,ZXDB
591,Q2QGD7,Q2QGD7,ZXDC
592,P14921-3,P14921,ETS1


In [9]:
known_ADs = pd.merge(known_ADs, known_AD_gene_names, left_on = "uniprotID", right_on = "From", how = "left")
known_ADs

,Gene,Start,End,uniprotID,Matching Isoforms,Canonical Transcript ID,orig_uniprotID,Reference,TileType,ProteinRegionSeq,Notes,From,Entry,Gene Names (primary)
0,ABRAXAS1,121,200,Q6UWZ7,[['Q6UWZ7']],ENST00000321945,Q6UWZ7,DelRosso et al.,CR,LQEHFSNQDLVFLLLTPSIITESCSTHRLEHSLYKPQKGLFHRVPL...,NaN,Q6UWZ7,Q6UWZ7,ABRAXAS1
1,AEBP1,1088,1158,Q8IUX7,[['Q8IUX7']],NaN,Q8IUX7,Staller Activity Data,TF,EVVTEFGTEVEPEFGTKVEPEFETQLEPEFETQLEPEFEEEEEEEK...,NaN,Q8IUX7,Q8IUX7,AEBP1
2,AHCTF1,1445,1698,Q8WYP5,"[['Q8WYP5'], ['Q8WYP5']]",nan / nan,Q8WYP5 / Q8WYP5,"PMID: 11952839, Soto / transcriptionalactivity...",TF,IRANDNKSMADVLGDGGNSSLTISEGPIVSERRLNQEVALNLKEDH...,NaN,Q8WYP5,Q8WYP5,AHCTF1
3,AHR,118,126,P35869,[['P35869']],NaN,P35869,"transcriptionalactivity_regions.txt, GSL",TF,LLQALNGFV,NaN,P35869,P35869,AHR
4,AHR,266,268,P35869,[['P35869']],NaN,P35869,"transcriptionalactivity_regions.txt, GSL",TF,FAI,NaN,P35869,P35869,AHR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,ZXDB,576,703,P98169,[['P98169']],NaN,P98169 / P98169,"R4TA_regions.txt / activation_regions.txt, GSL",TF,QDLLAQLEAANSLTPSSELTSQRQNDLSDAEIVSLFSDVPDSTSAA...,NaN,P98169,P98169,ZXDB
756,ZXDC,579,688,Q2QGD7,"[['Q2QGD7', 'Q2QGD7-2'], ['Q2QGD7', 'Q2QGD7-2']]",nan / nan,Q2QGD7 / Q2QGD7 / Q2QGD7,"PMID: 16600381, Soto / R4TA_regions.txt / acti...",TF,DSPLVLGTAATVLQQGSFSVDDVQTVSAGALGCLVALPMKNLSDDP...,NaN,Q2QGD7,Q2QGD7,ZXDC
757,c-ETS1,175,286,P14921-3,[['P14921-3']],NaN,P14921,"Choi 2000 list, GSL",TF,ILQKEDVKPYQVNGVNPAYPESRYTSDYFISYGIEHAQCVPPSEFS...,NaN,P14921-3,P14921,ETS1
758,c-Jun/AP-1,238,257,P05412,[['P05412']],NaN,P05412,"Choi 2000 list, GSL",TF,ETPPLSPIDMESQERIKAER,NaN,P05412,P05412,JUN


In [10]:
# Which of known ADs overlap SFARI Gene?

In [11]:
SFARI_Gene_known_ADs = pd.merge(known_ADs, SFARI, left_on = "Gene Names (primary)", right_on = "gene-symbol")
SFARI_Gene_known_ADs

,Gene,Start,End,uniprotID,Matching Isoforms,Canonical Transcript ID,orig_uniprotID,Reference,TileType,ProteinRegionSeq,...,status,gene-symbol,gene-name,ensembl-id,chromosome,genetic-category,gene-score,syndromic,eagle,number-of-reports
0,APBB1,221,330,O00213-2,[['O00213-2']],ENST00000311051,O00213,DelRosso et al.,CR,DSSWATLSQGSPSYGSPEDTDSFWNPNAFETDSDLPAGWMRVQDTS...,...,9,APBB1,amyloid beta precursor protein binding family ...,ENSG00000166313,11,"Rare Single Gene Mutation, Functional",2.0,0,NaN,4
1,AR,652,910,P10275,[['P10275']],NaN,P10275,"PMID: 9033395, Soto",TF,PTEETTQKLTVSHIEGYECQPIFLNVLEAIEPGVVCAGHDNNQPDS...,...,9,AR,androgen receptor,NaN,X,Genetic Association,2.0,0,NaN,6
2,AR,1,494,P10275,"[['P10275', 'P10275-3', 'P10275-4']]",NaN,P10275,"PMID: 9033395, 7706276, Soto",TF,MEVQLGLGRVYPRPPSKTYRGAFQNLFQSVREVIQNPGPRHPEAAS...,...,9,AR,androgen receptor,NaN,X,Genetic Association,2.0,0,NaN,6
3,ARNT2,524,717,Q9HBZ2,[['Q9HBZ2']],NaN,Q9HBZ2,"PMID: 8657146, Soto",TF,QGSPFPSGHSGKAFSSSVVHVPGVNDIQSSSSTGQNMSQISRQLNQ...,...,9,ARNT2,aryl-hydrocarbon receptor nuclear translocator 2,ENSG00000172379,15,"Rare Single Gene Mutation, Syndromic, Genetic ...",2.0,0,NaN,15
4,ARX,472,562,Q96QS3,[['Q96QS3']],NaN,Q96QS3,"PMID: 17331656, Soto",TF,RHPAFISPAFGRLFSTMAPLTSASTAAALLRQPTPAVEGAVASGAL...,...,9,ARX,aristaless related homeobox,ENSG00000004848,X,"Rare Single Gene Mutation, Syndromic",1.0,1,13.8,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,VDR,415,427,P11473,[['P11473']],NaN,P11473,"PMID: 15908514, Soto",TF,TPLVLEVFGNEIS,...,9,VDR,vitamin D receptor,ENSG00000111424,12,"Genetic Association, Functional",2.0,0,NaN,10
102,VDR,195,238,P11473,"[['P11473'], ['P11473']]",nan / nan,P11473 / P11473,"Choi 2000 list OR uniprot. check, GSL / Stalle...",TF,DMMDSSSFSNLDLSEEDSDDPSVTLELSQLSMLPHLADLVSYSI,...,9,VDR,vitamin D receptor,ENSG00000111424,12,"Genetic Association, Functional",2.0,0,NaN,10
103,YY1,371,397,P25490,[['P25490']],NaN,P25490,"activation_regions.txt, GSL",TF,RTHVRIHTGDRPYVCPFDGCNKKFAQS,...,9,YY1,YY1transcription factor,ENSG00000100811,14,"Rare Single Gene Mutation, Syndromic, Functional",1.0,1,NaN,7
104,YY1,1,69,P25490,[['P25490']],NaN,P25490,"PMID: 7731805, Soto",TF,MASGDTLYIATDGSEMPAEIVELHEIEVETIPVETIETTVVGEEEE...,...,9,YY1,YY1transcription factor,ENSG00000100811,14,"Rare Single Gene Mutation, Syndromic, Functional",1.0,1,NaN,7


In [29]:
SFARI_Gene_known_ADs.to_csv("../output/redone_ASD_assoc_known_ADs.csv")

In [48]:
lambert_TFs = pd.read_csv("../soto_analysis/data/SFARI_TFs_with_isoform_id.csv")
# lambert_TFs["uniprotID"] = lambert_TFs["GeneName"].str.split("|").str[1]
lambert_TFs

,Unnamed: 0,GeneName,Start,End,uniprotID,Reference,Sequence,matching_isoforms,isoforms,orig_uniprotID
0,0,ADNP,1,1102,Q9H2P0,NaN,MFQLPVNNLGSLRKARKTVKKILSDIGLEYCKEHIEDFKQFEPNDF...,['Q9H2P0'],1,Q9H2P0
1,1,AHDC1,1,1603,Q5TGY3,NaN,MRVKPQGLVVTSSAVCSSPDYLREPKYYPGGPPTPRPLLPTRPPAS...,['Q5TGY3'],1,Q5TGY3
2,2,ARID2,1,1835,Q68CP9-1,NaN,MANSTGKAPPDERRKGLAFLDELRQFHHSRGSPFKKIPAVGGKELD...,['Q68CP9'],1,Q68CP9
3,3,ARNT2,1,717,Q9HBZ2-1,NaN,MATPAAVNPPEMASDIPGSVTLPVAPMAATGQVRMAGAMPARGGKR...,['Q9HBZ2'],1,Q9HBZ2
4,4,ARX,1,562,Q96QS3,NaN,MSNQYQEEGCSERPECKSKSPTLLSSYCIDSILGRRSPCKMRLLGA...,['Q96QS3'],1,Q96QS3
...,...,...,...,...,...,...,...,...,...,...
122,122,ZNF711,1,761,Q9Y462-1,NaN,MDSGGGSLGLHTPDSRMAHTMIMQDFVAGMAGTAHIDGDHIVVSVP...,['Q9Y462'],1,Q9Y462
123,123,ZNF713,1,430,no match,NaN,MEEEEMNDGSQMVRSQESLTFQDVAVDFTREEWDQLYPAQKNLYRD...,[],0,Q8N859
124,124,ZNF774,1,483,Q6NX45,NaN,MWLGTSGKSGLPGHCLENPLQECHPAQLEEWALKGISRPSVISQPE...,['Q6NX45'],1,Q6NX45
125,125,ZNF804A,1,1209,Q7Z570,NaN,MECYYIVISSTHLSNGHFRNIKGVFRGPLSKNGNKTLDYAEKENTI...,['Q7Z570'],1,Q7Z570


In [49]:
# Maps uniprotIDs to isoforms

def get_isoform_ids(df):
    isoform_mappings_df = pd.read_csv("../soto_analysis/data/SFARI_TFs_all_isoforms_mapped.csv")
    uniprotID_mappings = {}
    for key, val in zip(isoform_mappings_df["From"], isoform_mappings_df["uniprotID"]):
        uniprotID_mappings[key] = val
    return df.replace({"uniprotID": uniprotID_mappings}) 

In [50]:
SFARI_Gene_known_ADs_isoforms = get_isoform_ids(SFARI_Gene_known_ADs)
SFARI_Gene_known_ADs_isoforms

,Gene,Start,End,uniprotID,Matching Isoforms,Canonical Transcript ID,orig_uniprotID,Reference,TileType,ProteinRegionSeq,...,status,gene-symbol,gene-name,ensembl-id,chromosome,genetic-category,gene-score,syndromic,eagle,number-of-reports
0,APBB1,221,330,O00213-2,[['O00213-2']],ENST00000311051,O00213,DelRosso et al.,CR,DSSWATLSQGSPSYGSPEDTDSFWNPNAFETDSDLPAGWMRVQDTS...,...,9,APBB1,amyloid beta precursor protein binding family ...,ENSG00000166313,11,"Rare Single Gene Mutation, Functional",2.0,0,NaN,4
1,AR,652,910,P10275-1,[['P10275']],NaN,P10275,"PMID: 9033395, Soto",TF,PTEETTQKLTVSHIEGYECQPIFLNVLEAIEPGVVCAGHDNNQPDS...,...,9,AR,androgen receptor,NaN,X,Genetic Association,2.0,0,NaN,6
2,AR,1,494,P10275-1,"[['P10275', 'P10275-3', 'P10275-4']]",NaN,P10275,"PMID: 9033395, 7706276, Soto",TF,MEVQLGLGRVYPRPPSKTYRGAFQNLFQSVREVIQNPGPRHPEAAS...,...,9,AR,androgen receptor,NaN,X,Genetic Association,2.0,0,NaN,6
3,ARNT2,524,717,Q9HBZ2-1,[['Q9HBZ2']],NaN,Q9HBZ2,"PMID: 8657146, Soto",TF,QGSPFPSGHSGKAFSSSVVHVPGVNDIQSSSSTGQNMSQISRQLNQ...,...,9,ARNT2,aryl-hydrocarbon receptor nuclear translocator 2,ENSG00000172379,15,"Rare Single Gene Mutation, Syndromic, Genetic ...",2.0,0,NaN,15
4,ARX,472,562,Q96QS3,[['Q96QS3']],NaN,Q96QS3,"PMID: 17331656, Soto",TF,RHPAFISPAFGRLFSTMAPLTSASTAAALLRQPTPAVEGAVASGAL...,...,9,ARX,aristaless related homeobox,ENSG00000004848,X,"Rare Single Gene Mutation, Syndromic",1.0,1,13.8,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,VDR,415,427,P11473-1,[['P11473']],NaN,P11473,"PMID: 15908514, Soto",TF,TPLVLEVFGNEIS,...,9,VDR,vitamin D receptor,ENSG00000111424,12,"Genetic Association, Functional",2.0,0,NaN,10
102,VDR,195,238,P11473-1,"[['P11473'], ['P11473']]",nan / nan,P11473 / P11473,"Choi 2000 list OR uniprot. check, GSL / Stalle...",TF,DMMDSSSFSNLDLSEEDSDDPSVTLELSQLSMLPHLADLVSYSI,...,9,VDR,vitamin D receptor,ENSG00000111424,12,"Genetic Association, Functional",2.0,0,NaN,10
103,YY1,371,397,P25490,[['P25490']],NaN,P25490,"activation_regions.txt, GSL",TF,RTHVRIHTGDRPYVCPFDGCNKKFAQS,...,9,YY1,YY1transcription factor,ENSG00000100811,14,"Rare Single Gene Mutation, Syndromic, Functional",1.0,1,NaN,7
104,YY1,1,69,P25490,[['P25490']],NaN,P25490,"PMID: 7731805, Soto",TF,MASGDTLYIATDGSEMPAEIVELHEIEVETIPVETIETTVVGEEEE...,...,9,YY1,YY1transcription factor,ENSG00000100811,14,"Rare Single Gene Mutation, Syndromic, Functional",1.0,1,NaN,7


In [51]:
SFARI_Gene_known_ADs_on_TFs = SFARI_Gene_known_ADs_isoforms[SFARI_Gene_known_ADs_isoforms["uniprotID"].isin(lambert_TFs["uniprotID"])]
SFARI_Gene_known_ADs_on_TFs

,Gene,Start,End,uniprotID,Matching Isoforms,Canonical Transcript ID,orig_uniprotID,Reference,TileType,ProteinRegionSeq,...,status,gene-symbol,gene-name,ensembl-id,chromosome,genetic-category,gene-score,syndromic,eagle,number-of-reports
1,AR,652,910,P10275-1,[['P10275']],NaN,P10275,"PMID: 9033395, Soto",TF,PTEETTQKLTVSHIEGYECQPIFLNVLEAIEPGVVCAGHDNNQPDS...,...,9,AR,androgen receptor,NaN,X,Genetic Association,2.0,0,NaN,6
2,AR,1,494,P10275-1,"[['P10275', 'P10275-3', 'P10275-4']]",NaN,P10275,"PMID: 9033395, 7706276, Soto",TF,MEVQLGLGRVYPRPPSKTYRGAFQNLFQSVREVIQNPGPRHPEAAS...,...,9,AR,androgen receptor,NaN,X,Genetic Association,2.0,0,NaN,6
3,ARNT2,524,717,Q9HBZ2-1,[['Q9HBZ2']],NaN,Q9HBZ2,"PMID: 8657146, Soto",TF,QGSPFPSGHSGKAFSSSVVHVPGVNDIQSSSSTGQNMSQISRQLNQ...,...,9,ARNT2,aryl-hydrocarbon receptor nuclear translocator 2,ENSG00000172379,15,"Rare Single Gene Mutation, Syndromic, Genetic ...",2.0,0,NaN,15
4,ARX,472,562,Q96QS3,[['Q96QS3']],NaN,Q96QS3,"PMID: 17331656, Soto",TF,RHPAFISPAFGRLFSTMAPLTSASTAAALLRQPTPAVEGAVASGAL...,...,9,ARX,aristaless related homeobox,ENSG00000004848,X,"Rare Single Gene Mutation, Syndromic",1.0,1,13.8,24
9,C2D1A,22,60,Q6P1N0-1,"[['Q6P1N0', 'Q6P1N0-2']]",NaN,Q6P1N0,Staller Activity Data,TF,GLLVDLSPDGLMIPEDGANDEELEAEFLALVGGQPPALE,...,9,CC2D1A,Coiled-coil and C2 domain containing 1A,ENSG00000132024,19,"Rare Single Gene Mutation, Functional",2.0,0,NaN,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,THRA,1,52,P10827-1,"[['P10827', 'P10827-2', 'P10827-3', 'P10827-4']]",NaN,P10827,"PMID: 27347890, Soto",TF,MEQKPSKVECGSDPEENSARSPDGKRKRKNGQCSLKTSMSGYIPSY...,...,9,THRA,thyroid hormone receptor alpha,ENSG00000126351,17,"Rare Single Gene Mutation, Functional",2.0,0,NaN,5
101,VDR,415,427,P11473-1,[['P11473']],NaN,P11473,"PMID: 15908514, Soto",TF,TPLVLEVFGNEIS,...,9,VDR,vitamin D receptor,ENSG00000111424,12,"Genetic Association, Functional",2.0,0,NaN,10
102,VDR,195,238,P11473-1,"[['P11473'], ['P11473']]",nan / nan,P11473 / P11473,"Choi 2000 list OR uniprot. check, GSL / Stalle...",TF,DMMDSSSFSNLDLSEEDSDDPSVTLELSQLSMLPHLADLVSYSI,...,9,VDR,vitamin D receptor,ENSG00000111424,12,"Genetic Association, Functional",2.0,0,NaN,10
103,YY1,371,397,P25490,[['P25490']],NaN,P25490,"activation_regions.txt, GSL",TF,RTHVRIHTGDRPYVCPFDGCNKKFAQS,...,9,YY1,YY1transcription factor,ENSG00000100811,14,"Rare Single Gene Mutation, Syndromic, Functional",1.0,1,NaN,7


In [52]:
SFARI_Gene_known_ADs_on_TFs.to_csv("../output/redone_ASD_assoc_known_ADs_on_TFs.csv")